# W205 Project 2 Tracking User Activity

### Background:

I work as a data scientst at an ed tech firm. In order to find business insights, I am going to explore assessments that lots of our clients are using and try to find the patterns of the data.  

### read the file

Identify my current work directory

In [1]:
! pwd

/project-2-fengyaoluo


Set my work directory as /project-2-fengyaoluo/

In [2]:
! cd ~/project-2-fengyaoluo/

/bin/sh: 1: cd: can't cd to /root/project-2-fengyaoluo/


Read the file from the link and output it as a json file

In [3]:
! curl -L -o assessment-nested.json https://goo.gl/ME6hjp

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 9096k  100 9096k    0     0  9292k      0 --:--:-- --:--:-- --:--:-- 1714k


### Turn on Docker Compose

Turn up the docker compose 

In [5]:
! docker-compose up -d

/bin/sh: 1: docker-compose: not found


Check what is in Cloudera hadoop

In [6]:
! docker-compose exec cloudera hadoop fs -ls /tmp/

/bin/sh: 1: docker-compose: not found


### Push data into kafka

Create the kafka topic "assessment"

In [7]:
! docker-compose exec kafka kafka-topics --create --topic assessment --partitions 1 --replication-factor 1 --if-not-exists --zookeeper zookeeper:32181

Created topic assessment.


If showing the error as "ERROR: No container found for kafka_1", try execute the following command. If not, skip.

`! docker-compose down`

### Parse Json file line by line to kafka

Use jq command to read the json file and -c to concatnate records in one line, then push into kafka, under the topic of assessment

In [8]:
! docker-compose exec mids bash -c "cat /project-2-fengyaoluo/assessment-nested.json | jq '.[]' -c | kafkacat -P -b kafka:29092 -t assessment"

### Run Pyspark

Use docker compose to execute pyspark (This will take a while)

In [ ]:
! docker-compose exec spark pyspark

Python 3.6.1 |Anaconda 4.4.0 (64-bit)| (default, May 11 2017, 13:09:58) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)] on linux
Type "help", "copyright", "credits" or "license" for more information.
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
20/10/21 03:10:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
20/10/21 03:10:28 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
20/10/21 03:10:28 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
20/10/21 03:10:33 ERROR Schema: Failed initialising database.
Unable to open a test connection to the given database. JDBC url = jdbc:derby:;databaseName=metastore_db;create=true, username = APP. Terminating connection pool (set lazyInit to true if you expe

Use spark to read the data in the topic "assessment" and name it as "raw_assessment"

In [ ]:
raw_assessment = spark.read.format("kafka").option("kafka.bootstrap.servers", "kafka:29092").option("subscribe","assessment").option("startingOffsets", "earliest").option("endingOffsets", "latest").load()  

Cache the raw_assessment therefore we won't encounter an error unless we evaluate it

In [ ]:
raw_assessment.cache()

Check the schema of the data

In [ ]:
raw_assessment.printSchema()

Cast the data's value as string and save it under the name as "assessment1"

### Cast raw data value to string and turn it into RDD

In [ ]:
! assessment1 = raw_assessment.select(raw_assessment.value.cast('string'))  

In [ ]:
! import sys

In [ ]:
! sys.stdout = open(sys.stdout.fileno(), mode='w', encoding='utf8', buffering=1)

In [ ]:
! import json

In [ ]:
! assessment1.rdd.map(lambda x: json.loads(x.value)).toDF().show()

In [ ]:
! from pyspark.sql import Row

In [ ]:
! extracted_commits.registerTempTable('assessment')

In [ ]:
! extracted_assessment.printSchema()

### Unnest the Json file

In [ ]:
questions_from_exam = spark.sql("select sequences.questions from assessment ").show(truncate = False)

In [ ]:
incomplete = spark.sql("select sequences.questions[1].user_incomplete from assessment").show(truncate = False)

### Analysis the data in spark

Question 1: How many assesstments are in the dataset?

In [ ]:
spark.sql("select count(*) from assessment ").show()

Question 2: What's the name of your Kafka topic? How did you come up with that name?

Question 3: How many people took Learning Git?

In [ ]:
spark.sql("select exam_name, count(*)  from assessment where exam_name = 'Learning Git' group by exam_name").show()

Question 4: What is the least common course taken? And the most common?

In [ ]:
spark.sql("select exam_name, count(*)  from assessment group by exam_name order by count(*) ").show(truncate = False)

In [ ]:
spark.sql("select exam_name, count(*)  from assessment group by exam_name order by count(*) desc ").show(truncate = False)

### Export the table as Parquet to HDFS

In [ ]:
questions_from_exam.write.parquet("/tmp/assessment")

### Turn down docker compose

In [ ]:
! docker-compose down